# Temperature nel tempo a Ostiglia (provincia di Mantova) — 2000 → 2025

## Obiettivo del notebook
Questo notebook costruisce una serie storica delle temperature a **Ostiglia (MN)** e mostra l’andamento nel tempo **dal 2000 al 2025**.

In particolare:
- scarichiamo i dati giornalieri (temperatura **media**, **minima**, **massima**) per le coordinate di Ostiglia;
- calcoliamo indicatori annuali (media annua della temperatura media giornaliera, e anche media delle minime e delle massime);
- disegniamo un grafico con:
  - asse **x** = anno (2000…2025)
  - asse **y** = temperatura (°C)

> **Nota metodologica (importante):** qui usiamo una sorgente “grigliata” (reanalysis) tramite l’**Open‑Meteo Historical Weather API**. I valori sono ottimi per analisi didattiche e trend nel tempo, ma non coincidono necessariamente al 100% con una stazione meteo locale specifica (dipende dalla distanza e dalla risoluzione del modello).  


## Fonte dati e scelta delle variabili

Useremo l’endpoint **/v1/archive** dell’Historical Weather API di Open‑Meteo, richiedendo variabili **giornaliere**:

- `temperature_2m_mean` (temperatura media giornaliera a 2 m)
- `temperature_2m_min` (temperatura minima giornaliera a 2 m)
- `temperature_2m_max` (temperatura massima giornaliera a 2 m)

Per le variabili giornaliere, la documentazione richiede anche un fuso orario (`timezone`) per definire correttamente i giorni nel calendario.

In questo notebook impostiamo:
- coordinate di Ostiglia (approssimazione WGS84)
- `timezone="Europe/Rome"`
- intervallo: **2000-01-01 → 2025-12-31**


## Librerie

- **requests**: per scaricare il JSON dall’API
- **pandas**: per gestire date, tabelle e raggruppamenti annuali
- **matplotlib**: per i grafici

Se `requests` non fosse installato nel tuo ambiente, puoi installarlo con:
```bash
pip install requests
```
(da terminale o da cella Jupyter con `!pip install requests`)


In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt


## Parametri: coordinate e intervallo temporale

Qui sotto impostiamo le coordinate di Ostiglia e le date di inizio/fine.
Se vuoi spostarti su un altro comune, basta cambiare latitudine e longitudine.


In [ ]:
# Coordinate di Ostiglia (MN) — valori indicativi
LAT = 45.067   # latitudine Nord
LON = 11.133   # longitudine Est

START_DATE = "2000-01-01"
END_DATE   = "2025-12-31"

TIMEZONE = "Europe/Rome"

LAT, LON, START_DATE, END_DATE


## Download dei dati (con cache locale)

Per evitare di riscaricare i dati ogni volta che riapri il notebook:
- se trova un file CSV locale, lo rilegge
- altrimenti chiama l’API e salva i dati su file

Questo è comodo anche a scuola: una volta scaricato, il notebook funziona più velocemente.


In [ ]:
from pathlib import Path

CACHE_FILE = Path("ostiglia_temperature_2000_2025.csv")

def fetch_open_meteo_daily(lat, lon, start_date, end_date, timezone="Europe/Rome"):
    """Scarica i dati giornalieri da Open-Meteo Historical Weather API."""
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "daily": "temperature_2m_mean,temperature_2m_min,temperature_2m_max",
        "timezone": timezone,
    }
    r = requests.get(url, params=params, timeout=60)
    r.raise_for_status()
    data = r.json()
    return data

def to_dataframe(payload):
    """Converte la risposta JSON in un DataFrame con colonna data e temperature."""
    daily = payload.get("daily", {})
    df = pd.DataFrame({
        "date": pd.to_datetime(daily.get("time")),
        "t_mean": daily.get("temperature_2m_mean"),
        "t_min": daily.get("temperature_2m_min"),
        "t_max": daily.get("temperature_2m_max"),
    })
    return df

if CACHE_FILE.exists():
    df = pd.read_csv(CACHE_FILE, parse_dates=["date"])
else:
    payload = fetch_open_meteo_daily(LAT, LON, START_DATE, END_DATE, TIMEZONE)
    df = to_dataframe(payload)
    df.to_csv(CACHE_FILE, index=False)

df.head(), df.tail(), df.shape


## Controllo rapido dei dati

Prima di fare grafici e statistiche, è buona pratica controllare:
- valori mancanti (`NaN`)
- intervallo di date effettivo


In [ ]:
df.isna().sum(), (df["date"].min(), df["date"].max())


# Costruzione della serie annuale (2000 → 2025)

## Metodo
Per ogni anno calcoliamo:
- media annua di `t_mean` (temperatura media giornaliera)
- media annua di `t_min` (media delle minime giornaliere)
- media annua di `t_max` (media delle massime giornaliere)

Formalmente, se \(T_{mean}(d)\) è la temperatura media del giorno \(d\), la media annua è:

\[
\overline{T}_{mean}(anno)=\frac{1}{N}\sum_{d=1}^{N} T_{mean}(d)
\]

dove \(N\) è il numero di giorni disponibili in quell’anno.


In [ ]:
df["year"] = df["date"].dt.year

annual = (
    df.groupby("year", as_index=False)
      .agg(t_mean_year=("t_mean", "mean"),
           t_min_year=("t_min", "mean"),
           t_max_year=("t_max", "mean"),
           n_days=("t_mean", "size"))
)

annual.head(), annual.tail()


## Grafico principale: temperatura annua (asse x = anni 2000…2025)

In questo grafico:
- la linea **t_mean_year** rappresenta la temperatura media annua (media delle medie giornaliere);
- le linee **t_min_year** e **t_max_year** aiutano a “contestualizzare” l’andamento (sono medie annue delle minime/massime giornaliere).

È un tipo di visualizzazione molto usata per discutere:
- variazioni interannuali
- possibili trend
- anni anomali (più caldi o più freddi rispetto al contesto)

> Nota: qui non stiamo facendo un’analisi statistica formale del trend (significatività, test, ecc.). Lo scopo è didattico e descrittivo.


In [ ]:
plt.figure(figsize=(12, 5))

plt.plot(annual["year"], annual["t_mean_year"], label="Media annua (t_mean)")
plt.plot(annual["year"], annual["t_min_year"],  label="Media annua delle minime (t_min)")
plt.plot(annual["year"], annual["t_max_year"],  label="Media annua delle massime (t_max)")

plt.title("Ostiglia (MN) — Andamento temperature annuali (2000–2025)")
plt.xlabel("Anno")
plt.ylabel("Temperatura (°C)")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


## (Opzionale) Evidenziare la “banda” tra minime e massime

Un modo visivo per rendere l’idea della variabilità è riempire l’area tra:
- media annua delle minime
- media annua delle massime

In questo caso l’area non rappresenta l’escursione termica giornaliera, ma l’andamento medio annuo dei due estremi giornalieri.


In [ ]:
plt.figure(figsize=(12, 5))

plt.plot(annual["year"], annual["t_mean_year"], label="Media annua (t_mean)")
plt.fill_between(
    annual["year"],
    annual["t_min_year"],
    annual["t_max_year"],
    alpha=0.2,
    label="Banda tra medie annue min/max"
)

plt.title("Ostiglia (MN) — Media annua + banda min/max (2000–2025)")
plt.xlabel("Anno")
plt.ylabel("Temperatura (°C)")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


# Andamento “dentro l’anno” (climatologia mensile) — utile per leggere la stagionalità

Il grafico annuale (2000…2025) ti mostra bene le variazioni tra anni, ma non ti fa “vedere” la stagionalità.

Una visualizzazione molto efficace è la **climatologia mensile**:
- raggruppiamo tutti i giorni per mese (gennaio, febbraio, …)
- calcoliamo la media sul periodo 2000–2025

Questo produce 12 valori e descrive una sorta di “anno medio” per Ostiglia.


In [ ]:
df["month"] = df["date"].dt.month

monthly_clim = (
    df.groupby("month", as_index=False)
      .agg(t_mean=("t_mean","mean"),
           t_min=("t_min","mean"),
           t_max=("t_max","mean"))
)

nomi_mesi = ["Gen","Feb","Mar","Apr","Mag","Giu","Lug","Ago","Set","Ott","Nov","Dic"]

plt.figure(figsize=(10, 4))
plt.plot(monthly_clim["month"], monthly_clim["t_mean"], label="Media")
plt.plot(monthly_clim["month"], monthly_clim["t_min"], label="Minima")
plt.plot(monthly_clim["month"], monthly_clim["t_max"], label="Massima")

plt.xticks(range(1,13), nomi_mesi)
plt.title("Ostiglia (MN) — Climatologia mensile media (2000–2025)")
plt.xlabel("Mese")
plt.ylabel("Temperatura (°C)")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


## Idee per attività didattiche (veloci)

1. **Anni più caldi e più freddi**
   - trova l’anno con `t_mean_year` massimo e minimo.
2. **Confronto tra due periodi**
   - ad esempio 2000–2012 vs 2013–2025: cambia davvero la media?
3. **Trend lineare**
   - aggiungi una regressione lineare e discuti cosa significa “pendenza”.
4. **Eventi estremi**
   - lavora sui massimi/minimi giornalieri (non mediati) per individuare ondate di caldo o gelo.


In [ ]:
# 1) anni più caldo/freddo (in termini di media annua)
anno_piu_caldo = annual.loc[annual["t_mean_year"].idxmax()]
anno_piu_freddo = annual.loc[annual["t_mean_year"].idxmin()]

anno_piu_caldo, anno_piu_freddo
